# Covid Brasil

In [ ]:
%config IPCompleter.greedy=True
#!pip install pandas
#!pip install seaborn
#!pip install plotly
#!pip install altair

In [5]:
import pandas as pd 
import numpy as np 
import seaborn as sns

from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML
import matplotlib.pyplot as plt 
import ipywidgets as widgets
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import altair as alt
print('Modules are imported.')

Modules are imported.


In [9]:
import io
import requests
url='https://data.brasil.io/dataset/covid19/caso.csv.gz'
s=requests.get(url).content


In [10]:
with open('data/covidio.csv.gz', 'wb') as f:
    f.write(s)

In [11]:
#df = pd.read_csv('data/covidio.csv.gz', parse_dates=['last_available_date'], index_col='last_available_date')
df = pd.read_csv('data/covidio.csv.gz')


In [ ]:
df.columns

In [12]:
df['mortalidade'] =  df['deaths']/df['estimated_population']
df['mortalidade100k'] = (df['deaths']*100000)/df['estimated_population']
df[['mortalidade', 'death_rate', 'mortalidade100k']].head()

,mortalidade,death_rate,mortalidade100k
0,0.001240,0.0137,124.046588
1,0.001237,0.0137,123.698468
2,0.001235,0.0137,123.466388
3,0.001229,0.0137,122.886189
4,0.001229,0.0138,122.886189


In [13]:
df_states = df[(df['is_last'] == True) &(df.place_type=='state')][['state','confirmed','deaths', 'death_rate','mortalidade','mortalidade100k']]
df_cities = df[(df['is_last'] == True) &(df.place_type=='city')][['city','state', 'confirmed','deaths', 'death_rate','mortalidade','mortalidade100k']]

In [14]:
# deletando linhas inuteis
df_cities.drop(df_cities[df_cities['city'] == 'Importados/Indefinidos'].index, inplace=True)

In [15]:
df_states.columns = ['Estado', 'Confirmados', 'Mortes', 'Letalidade','Mortalidade', 'Mortalidade100k']
df_cities.columns = ['Cidade', 'Estado', 'Confirmados', 'Mortes', 'Letalidade','Mortalidade', 'Mortalidade100k']

In [16]:
#df_cities.sort_values('Letalidade',ascending=False).head()
df_states['Letalidade_rank'] = df_states['Letalidade'].rank(method='min').astype(int)
df_states['Mortalidade_rank'] = df_states['Mortalidade'].rank(method='min').astype(int)
df_states['rank'] = ((df_states['Mortalidade_rank'] + df_states['Letalidade_rank'])/2).rank(method='min').astype(int)
df_cities['Letalidade_rank'] = df_cities['Letalidade'].rank(method='min').astype(int)
df_cities['Mortalidade_rank'] = df_cities['Mortalidade'].rank(method='min').astype(int)
df_cities['rank'] = ((df_cities['Mortalidade_rank'] + df_cities['Letalidade_rank'])/2).rank(method='min').astype(int)

In [36]:
def search_city(city, color='red', reverse=False):
    return df_cities[df['city'] == city]

def style_frame(df):
    cm = sns.light_palette(color, as_cmap=True, reverse=reverse)
    return (df.style.set_properties(**{'text-align': 'right'}).background_gradient(cmap=cm).hide_index()).set_caption(
    title)

def treemap_show(df=df_states, nivel='Estado', column='rank', n=10,  asc=False, title=""):
    fig = px.treemap(df.head(30), 
                 path=[nivel], 
                 values=column, height=700,
                 title=title,
                 color_discrete_sequence = px.colors.qualitative.Prism)

    fig.data[0].textinfo = 'label+text+value'
    #fig.write_image("images/fig2.png")

    fig.show()


def covid_data_table(df, nivel, column, n, asc=False,title="", color='red', reverse=False, corte=0):

    if(corte > 0):
        df = df[df['Mortes'] > corte]
    
    df = df.sort_values(column,ascending=asc).copy().head(n)

    data_table = pd.DataFrame()
    data_table[nivel] = df[nivel]
    if nivel == 'Cidade':
        data_table['Estado'] = df['Estado']
    #data_table["Confirmados"] = df['Confirmados']
    data_table["rank"] = df['rank']
    data_table["Letalidade_rank"] = df['Letalidade_rank']
    data_table["Mortalidade_rank"] = df['Mortalidade_rank']
    data_table["Mortes"] = df['Mortes']
    data_table["Mortalidade100k"] = df['Mortalidade100k'].astype(int)
    data_table["Letalidade"] = df['Letalidade']
    
    #data_table["Mortalidade"] = df['Mortalidade']
    
    cm = sns.light_palette(color, as_cmap=True, reverse=reverse)
    return (data_table.style.set_properties(**{'text-align': 'right'}).background_gradient(cmap=cm).hide_index()).set_caption(
    title)

def tabela_estado(column, n, asc=False,title="", color='red', reverse=False, corte=0):
    '''Retorna uma table para estado '''
    return covid_data_table(df_states,'Estado', column, n, asc=asc,title=title, color=color, reverse=reverse, corte=corte)

def tabela_estado_reversa(column, n, title="", corte=0):
    return tabela_estado(column, n, True, title, 'green', True, corte)

def tabela_cidade(column, n, asc=False,title="", color='red', reverse=False, corte=0):
    '''Retorna uma table para cidades '''
    return covid_data_table(df_cities,'Cidade', column, n, asc=asc,title=title, color=color, reverse=reverse, corte=corte)

def tabela_cidade_reversa(column, n, title="", corte=0):
    return tabela_cidade(column, n, True, title, 'green', True, corte=corte)

## Top 10 Piores no Ranking (Letalidade + Mortalidade)/2

In [25]:
tabela_estado('rank', 10)

Estado,rank,Letalidade_rank,Mortalidade_rank,Mortes,Mortalidade100k,Letalidade
RJ,27,27,26,30172,173,0.057000
AM,26,25,27,8564,203,0.031300
MT,25,22,24,5214,147,0.023500
SP,23,24,18,53704,116,0.029700
PE,23,26,16,10399,108,0.039200
CE,22,23,17,10556,114,0.027900
ES,21,13,23,5929,145,0.019900
SE,20,14,19,2801,120,0.020100
GO,19,17,15,7599,106,0.021300
DF,17,5,25,4582,149,0.016400


## Top 10 Melhores Estados no Ranking

In [26]:
tabela_estado_reversa('rank',10)

Estado,rank,Letalidade_rank,Mortalidade_rank,Mortes,Mortalidade100k,Letalidade
TO,1,3,5,1397,87,0.013500
BA,1,6,2,10216,68,0.017100
SC,3,1,8,6445,88,0.011000
PR,4,10,6,10190,88,0.018200
MG,5,15,3,15315,71,0.020500
AC,6,7,12,879,98,0.017700
MA,6,18,1,4730,66,0.022700
PI,8,11,9,3092,94,0.019200
MS,9,9,14,2958,105,0.018100
RS,9,12,11,10826,94,0.019500


## Top  20 piores cidades do brasil com +1000 Obitos

In [32]:
tabela_cidade('rank',20, corte=500)

Cidade,Estado,rank,Letalidade_rank,Mortalidade_rank,Mortes,Mortalidade100k,Letalidade
Rio de Janeiro,RJ,5569,5525,5551,17352,257,0.091500
Manaus,AM,5523,5198,5544,5490,247,0.045900
São Vicente,SP,5504,5350,5321,587,159,0.055200
Recife,PE,5499,5246,5415,2906,175,0.048500
Várzea Grande,MT,5488,5107,5514,615,213,0.043000
São João de Meriti,RJ,5484,5546,5071,634,134,0.116000
Guarujá,SP,5481,5233,5377,543,168,0.047600
Jaboatão dos Guararapes,PE,5470,5399,5166,998,141,0.060400
Osasco,SP,5466,5276,5271,1066,152,0.050100
Olinda,PE,5462,5257,5275,600,152,0.049100


## Top 20 Cidades do Brasil

In [39]:
tabela_cidade_reversa('Mortalidade',20,corte=5)

Cidade,Estado,rank,Letalidade_rank,Mortalidade_rank,Mortes,Mortalidade100k,Letalidade
Rio Negrinho,SC,457,460,553,6,14,0.003500
Tuntum,MA,473,513,557,6,14,0.004200
Santana do Araguaia,PA,521,639,573,11,14,0.005700
São Félix do Xingu,PA,579,772,615,21,15,0.006800
Monte Santo,BA,1285,2396,626,8,16,0.015500
Santa Maria da Boa Vista,PE,1162,2108,639,7,16,0.014100
Parnarama,MA,638,873,656,6,17,0.007500
Seabra,BA,498,442,698,8,18,0.003200
Cururupu,MA,757,1111,712,6,18,0.009000
Laranjeiras do Sul,PR,641,812,721,6,18,0.007100


## Top 10 menor letalidade 

In [ ]:
covid_data_table_reverse(df_states,'Letalidade', 10,'Top 10 Menor Letalidade')

In [ ]:
def covid_bubble_chart(df, column, n, asc=False):
    df = df.sort_values(column,ascending=asc).copy().head(n)
    fig = px.scatter(df, x="Estado", y=column, size=column, color="Estado",
               text="Estado", size_max=90)
    fig.update_layout(
    title="Os top " + str(n) + ' piores estados por '  + column,
    xaxis_title="Estados",
    yaxis_title=column,
    width = 700
    )
    fig.show();
    
covid_bubble_chart(df_states, 'Letalidade',10, True)


In [ ]:
covid_bubble_chart(df_states, 'Mortes',10)